  Author: Ankit Kariryaa, University of Bremen
  
  Modified by Xuehui Pi and Qiuqi Luo

### Getting started
Define the paths for the dataset and trained models in the `notebooks/config/UNetTraining.py` file.  

In [1]:
import os
os.environ["MKL_NUM_THREADS"] = '16'
os.environ["NUMEXPR_NUM_THREADS"] = '16'
os.environ["OMP_NUM_THREADS"] = '16'
print(os.environ.get('OMP_NUM_THREADS'))

16


In [2]:
import tensorflow as tf
import numpy as np
from PIL import Image
import rasterio
import imgaug as ia
from imgaug import augmenters as iaa
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import mixed_precision 
mixed_precision.set_global_policy('mixed_float16')

import imageio
import os

import time
import rasterio.warp             # Reproject raster samples
from functools import reduce
from tensorflow.keras.models import load_model

from core.UNet_572 import UNet  #
from core.losses import tversky, focalTversky, bce_dice_loss, accuracy, dice_loss, IoU, recall, precision,F1_score
# from tensorflow.keras.losses import BinaryCrossentropy as bce
from core.optimizers import adaDelta, adagrad, adam, nadam
from core.frame_info_test import FrameInfo
from core.dataset_generator_test import DataGenerator
from core.visualize import display_images,plot
# from core.plot import display_images,plot
import json
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import pickle
import random

%matplotlib inline
import matplotlib.pyplot as plt  # plotting tools
import matplotlib.patches as patches
from matplotlib.patches import Polygon
#matplotlib.use("Agg")

import warnings                  # ignore annoying warnings
warnings.filterwarnings("ignore")
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

%reload_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


#Mixed precision is the use of both 16-bit and 32-bit floating-point types in a model during training to make it run faster and use less memory.
os.environ['TF_ENABLE_AUTO_MIXED_PRECISION'] = '1'
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
print(tf.__version__)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090, compute capability 8.6
2.9.0


In [3]:
# print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
# tf.device('/gpu:1')

In [4]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto(
    #device_count={"CPU": 64},
    allow_soft_placement=True, 
    log_device_placement=False)
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [5]:
# Required configurations (including the input and output paths) are stored in a separate file (such as config/UNetTraining.py)
# Please provide required info in the file before continuing with this notebook. 
# hbh: in this scene,a new config named UNetTraining_sequential is created to distinguish from the original
from config import UNetTraining_test
# In case you are using a different folder name such as configLargeCluster, then you should import from the respective folder 
# Eg. from configLargeCluster import UNetTraining
config = UNetTraining_test.Configuration()

In [6]:
def readImgs(path_to_write, fn):
    image = rasterio.open(os.path.join(path_to_write, fn))
#     print(fn)
    read_image = image.read()/1000
#     print(read_image[1].max())
    
    comb_img = np.transpose(read_image, axes=(1,2,0))
    annotation_im = Image.open(os.path.join(path_to_write, fn.replace(config.image_fn,config.annotation_fn).replace(config.image_type,config.ann_type)))
    annotation = np.array(annotation_im)
    rowNum=comb_img.shape[0]/config.patch_size[0]
    colNum=comb_img.shape[1]/config.patch_size[1]
    if(rowNum<1)or(colNum<1):
        print(fn)
#     if(read_image.shape[1]- annotation.shape[0]!=184) or(read_image.shape[2]- annotation.shape[1]!=184):
#                 print(fn)
    f = FrameInfo(comb_img, annotation)
    return f,rowNum*colNum
    
def readFrames(dataType):
    frames=[]
    numList=[]
    print(dataType)
#     for i in range(3,4):
    for i in range(0,config.type_num):#config.type_num
        path_to_write=os.path.join(config.dataset_dir,'{}/type{}'.format(dataType,i))
        all_files = os.listdir(path_to_write)
        all_files_image = [fn for fn in all_files if fn.startswith(config.image_fn) and fn.endswith(config.image_type)]#image.png
        print('type{} image number:{}'.format(i,len(all_files_image)))
        for j, fn in enumerate(all_files_image):
            f,num = readImgs(path_to_write,fn)
            frames.append(f)
            numList.append(num)
        # sub=np.array(numList)
        # print('{},{}'.format(len(all_files_image),sub.sum()))
    return frames,numList


## 参数初始化

In [7]:
OPTIMIZER = adaDelta
OPTIMIZER =  mixed_precision.LossScaleOptimizer(OPTIMIZER)
OPTIMIZER_NAME = 'AdaDelta'

# OPTIMIZER = adam
# OPTIMIZER = mixed_precision.LossScaleOptimizer(OPTIMIZER)
# OPTIMIZER_NAME = 'adam'

In [8]:
# LOSS = tversky 
# LOSS_NAME = 'tversky'

# LOSS=focalTversky
# LOSS_NAME = 'focalTversky'

#LOSS=tf.keras.losses.BinaryCrossentropy()
#LOSS_NAME = 'bce'

# LOSS=bce_dice_loss
# LOSS_NAME = 'bce_dice_loss'

LOSS=dice_loss
LOSS_NAME = 'dice_loss'


## 数据集准备

In [9]:
frames,numList=readFrames('train')
percentages=np.array(numList)
train_sum=percentages.sum()
percentages=percentages/percentages.sum()
print('total training img count:'+str(len(frames)))
print('total training patches count:{}'.format(train_sum))
train_generator = DataGenerator(config.input_size, config.output_size,config.step_size,frames, augmenter = 'iaa').random_generator(config.BATCH_SIZE,percentages)#,normalize =

train
type0 image number:156
type1 image number:141
image_8671.tif
type2 image number:68
type3 image number:53
type4 image number:9
type5 image number:17
total training img count:444
total training patches count:10000.691463470459


In [10]:
# numList=readNum('train')
# percentages=np.array(numList)
# print(percentages.sum())

In [11]:
frames,numList=readFrames('val')
percentages=np.array(numList)
val_sum=percentages.sum()
percentages=percentages/percentages.sum()
print('total validation img count:'+str(len(frames)))
print('total validation patches count:{}'.format(val_sum))
val_generator = DataGenerator(config.input_size, config.output_size,config.patch_size, frames, augmenter = None).random_generator(config.BATCH_SIZE,percentages)#, normalize = config.normalize

val
type0 image number:53
type1 image number:48
type2 image number:24
type3 image number:18
type4 image number:3
type5 image number:6
total validation img count:152
total validation patches count:3329.504222869873


In [12]:
# print(numList)

In [13]:
del frames

In [ ]:
for _ in range(1):
    train_images, real_label = next(train_generator) 
    print(train_images.shape)
    print(real_label.shape)
    display_images(train_images,real_label,pad=92,output_size=324)

In [ ]:
for _ in range(1):
    val_images, val_label = next(val_generator) 
    print(val_images.shape)
    display_images(val_images,val_label,pad=92,output_size=324)

### 训练参数设置

In [16]:
timestr = time.strftime("%Y%m%d-%H%M")
chf = config.input_image_channel + config.input_label_channel
chs = reduce(lambda a,b: a+str(b), chf, '') 

if not os.path.exists(config.model_path):
    os.makedirs(config.model_path)
model_name='_{}_{}_{}_{}_{}_test.h5'.format(timestr,OPTIMIZER_NAME,LOSS_NAME,chs,config.input_shape[0])
model_path = os.path.join(config.model_path,'lakes'+model_name)

chf = config.input_image_channel + config.input_label_channel
chs = reduce(lambda a,b: a+str(b), chf, '') 
print(model_path)

G:\5_lakemapping\5_saved_models\lakes_20240108-1240_AdaDelta_dice_loss_012345_508_test.h5


In [14]:
model_name='_20240108-1240_AdaDelta_dice_loss_012345_508_test.h5'
model_path=r'G:\5_lakemapping\5_saved_models\lakes_20240108-1240_AdaDelta_dice_loss_012345_508_test.h5'

In [23]:
# Define the model and compile it  
print(config.input_shape)
shape=[config.BATCH_SIZE, *config.input_shape]
model = UNet([config.BATCH_SIZE, *config.input_shape],config.input_label_channel)
model.load_weights(r'G:\5_lakemapping\5_saved_models\lakes_20240108-1240_AdaDelta_dice_loss_012345_508_test.h5')
# model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[dice_loss, accuracy, recall, precision,F1_score, IoU])
model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[accuracy, recall, precision,F1_score, IoU])

(508, 508, 5)
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 508, 508, 5  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_57 (Conv2D)             (None, 506, 506, 64  2944        ['Input[0][0]']                  
                                )                                                                 
                                                                                                  
 conv2d_58 (Conv2D)             (None, 504, 504, 64  36928       ['conv2d_57[0][0]']              
                                )                                             

                                                                                                  
 conv2d_69 (Conv2D)             (None, 86, 86, 256)  1769728     ['concatenate_13[0][0]']         
                                                                                                  
 conv2d_70 (Conv2D)             (None, 84, 84, 256)  590080      ['conv2d_69[0][0]']              
                                                                                                  
 up_sampling2d_14 (UpSampling2D  (None, 168, 168, 25  0          ['conv2d_70[0][0]']              
 )                              6)                                                                
                                                                                                  
 batch_normalization_30 (BatchN  (None, 168, 168, 25  1024       ['up_sampling2d_14[0][0]']       
 ormalization)                  6)                                                                
          

In [16]:
# Define callbacks      for the early stopping of training, LearningRateScheduler and model checkpointing 
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard

checkpoint = ModelCheckpoint(model_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = False)

#reduceonplatea： It can be useful when using adam as optimizer
#Reduce learning rate when a metric has stopped improving (after some patience 个epoch, reduce by a factor of 0.33, new_lr = lr * factor). 
#cooldown: number of epochs to wait before resuming normal operation after lr has been reduced. 

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.33,
                                   patience=4, verbose=1, mode='min',
                                   min_delta=0.0001, cooldown=4, min_lr=1e-16) 

early = EarlyStopping(monitor="val_loss", mode="min", verbose=2, patience=40)


log_dir = os.path.join('./logs','UNet'+model_name)
tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')

callbacks_list = [checkpoint, tensorboard] #reduceLROnPlat is not required with adaDelta, early

## 训练模型
保存history文件并绘图

In [24]:
loss_history = model.fit(train_generator, 
                         steps_per_epoch=train_sum//config.BATCH_SIZE,#config.MAX_TRAIN_STEPS,
                         epochs=config.NB_EPOCHS, 
                         validation_data=val_generator,
                         validation_steps=val_sum//config.BATCH_SIZE,#config.VALID_IMG_COUNT,
                         callbacks=callbacks_list,
                         workers=1,
                         initial_epoch=81
                         # shuffle=True,
#                          use_multiprocessing=True # the generator is not very thread safe 
                         #max_queue_size = 60,
                        )
h=loss_history.history
with open('history_{}_{}_{}_{}_{}.txt'.format(timestr,OPTIMIZER_NAME,LOSS_NAME, chs,config.input_shape[0]), 'wb') as file_pi:
    pickle.dump(h, file_pi)
plot(h,timestr, OPTIMIZER_NAME,LOSS_NAME, config.input_size[0], config.NB_EPOCHS, config.BATCH_SIZE,chs)

Epoch 82/150
625/625 [==============================] - ETA: 0s - loss: 0.0666 - accuracy: 0.9963 - recall: 0.9393 - precision: 0.9325 - F1_score: 0.9335 - IoU: 0.8791
Epoch 82: val_loss improved from inf to 0.11151, saving model to G:\5_lakemapping\5_saved_models\lakes_20240108-1240_AdaDelta_dice_loss_012345_508_test.h5
625/625 [==============================] - 292s 463ms/step - loss: 0.0666 - accuracy: 0.9963 - recall: 0.9393 - precision: 0.9325 - F1_score: 0.9335 - IoU: 0.8791 - val_loss: 0.1115 - val_accuracy: 0.9936 - val_recall: 0.8649 - val_precision: 0.9290 - val_F1_score: 0.8886 - val_IoU: 0.8131
Epoch 83/150
625/625 [==============================] - ETA: 0s - loss: 0.0740 - accuracy: 0.9962 - recall: 0.9283 - precision: 0.9300 - F1_score: 0.9261 - IoU: 0.8681
Epoch 83: val_loss improved from 0.11151 to 0.10124, saving model to G:\5_lakemapping\5_saved_models\lakes_20240108-1240_AdaDelta_dice_loss_012345_508_test.h5
625/625 [==============================] - 291s 465ms/step 

KeyboardInterrupt: 

In [14]:
##### 读取现有history文件
with open('/home/nkd/hbh/U_Net/notebooks/history/6type_748/history_20240107-1503_AdaDelta_dice_loss_012345_508.txt','rb')as file_pi: 
    h=pickle.load(file_pi)
print(h)

{'loss': [0.2703271508216858, 0.23761411011219025, 0.2186886966228485, 0.2124900072813034, 0.18981969356536865, 0.16862866282463074, 0.1608177125453949, 0.15647351741790771, 0.1472724974155426, 0.134345144033432, 0.12918660044670105, 0.13228817284107208, 0.12346545606851578, 0.11509436368942261, 0.11709623783826828, 0.1086016446352005, 0.10827430337667465, 0.10800683498382568, 0.10149221867322922, 0.10356645286083221, 0.10149320960044861, 0.10315406322479248, 0.0958845391869545, 0.09526585042476654, 0.09354756772518158, 0.08889814466238022, 0.08547293394804001, 0.09056983888149261, 0.08775176852941513, 0.09133173525333405, 0.09033767879009247, 0.08415089547634125, 0.08325526863336563, 0.08176227658987045, 0.08084473013877869, 0.08245421200990677, 0.0811118558049202, 0.07935318350791931, 0.08054818958044052, 0.0833011344075203, 0.08196241408586502, 0.08355109393596649, 0.08282841742038727, 0.0773802101612091, 0.07471518963575363, 0.07676729559898376, 0.07393141835927963, 0.0694467872381

In [ ]:
chs='012345'
timestr='20240107-1503'
plot(h,timestr, OPTIMIZER_NAME,LOSS_NAME, config.input_size[0], config.NB_EPOCHS, config.BATCH_SIZE,chs)

## 模型精度评价

In [9]:
# Load model after training 
model_path=os.path.join(config.model_path,'lakes_20240110-2311_AdaDelta_dice_loss_012345_572.h5')
model = load_model(model_path, custom_objects={'dice loss': LOSS, 'accuracy':accuracy ,'recall':recall, 'F1_score':F1_score,'precision':precision,'IoU': IoU}, compile=False) 
model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=[dice_loss, accuracy,recall, precision, IoU])

In [10]:
# # Print one batch on the training/test data! 
# for i in range(1):
#     test_images, real_label = next(test_generator)
#     #3 images per row: GSW, label, prediction
#     prediction = model.predict(test_images, steps=1)
#     prediction[prediction>0.5]=1
#     prediction[prediction<=0.5]=0
#     display_images(np.concatenate((test_images, real_label, prediction), axis = -1))# test_images( NDWI), real_label(label), prediction

In [30]:
j=4
frames=[]
path_to_write=os.path.join(config.dataset_dir,'test/type'+str(j))
all_files = os.listdir(path_to_write)
all_image_files = [fn for fn in all_files if fn.startswith(config.image_fn) and fn.endswith(config.image_type)]#ndwi.png
for i, fn in enumerate(all_image_files):
    f,nums = readImgs(path_to_write,fn)
    frames.append(f)
test_DGT=DataGenerator(config.input_size, config.output_size,config.patch_size,frames, augmenter = None)
tp_img,tp_ann= test_DGT.all_sequential_patches(config.step_size)
print(len(tp_ann))
titles=['ndwi','rgb','swir','annotation','prediction']
step=8
for i in range(len(tp_img)//step+1):
    prediction = model.predict(tp_img[i*step:(i+1)*step], steps=1)
    prediction[prediction>0.5]=1
    prediction[prediction<=0.5]=0
    fn=r'D:\sample746\image\type{}_{}_572.png'.format(j,i)
    display_images(tp_img[i*step:(i+1)*step], tp_ann[i*step:(i+1)*step], prediction,92,config.output_size[0],fn,titles=titles)

16
1/1 [==============================] - 0s 27ms/step


ZeroDivisionError: division by zero

## 总体精度评价

In [11]:
frames,numList=readFrames('test')
print(sum(numList))
test_patches = DataGenerator(config.input_size, config.output_size,config.patch_size, frames, augmenter = None).all_sequential_patches(config.step_size)
print('test patchs number:',len(test_patches[0]))

test
type0 image number:53
type1 image number:48
type2 image number:23
type3 image number:18
type4 image number:3
type5 image number:6
3530.4445838928223
test patchs number: 2881


In [12]:
ev=model.evaluate(test_patches[0],test_patches[1],config.BATCH_SIZE)

2024-01-08 09:49:05.921961: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 14869647680 exceeds 10% of free system memory.
2024-01-08 09:49:16.648040: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-01-08 09:49:16.667102: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3000000000 Hz
2024-01-08 09:49:18.711711: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8
2024-01-08 09:49:23.413495: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-01-08 09:49:23.414357: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Unimplemented: /usr/local/cuda-11.0/bin/ptx

  2/181 [..............................] - ETA: 19s - loss: 0.0927 - dice_loss: 0.0926 - accuracy: 0.9957 - recall: 0.8502 - precision: 0.9849 - IoU: 0.8375    

2024-01-08 09:50:43.141683: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'



180/181 [============================>.] - ETA: 0s - loss: 0.0937 - dice_loss: 0.0936 - accuracy: 0.9956 - recall: 0.9120 - precision: 0.9166 - IoU: 0.8395

2024-01-08 09:51:04.536050: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-01-08 09:51:04.984910: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-01-08 09:51:05.438124: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-01-08 09:51:05.854871: W tensorflow/stream_executor/gpu/

181/181 [==============================] - 171s 471ms/step - loss: 0.0940 - dice_loss: 0.0940 - accuracy: 0.9956 - recall: 0.9117 - precision: 0.9162 - IoU: 0.8391


2024-01-08 09:52:07.920786: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'



In [19]:
print(ev)

[0.12233785539865494, 0.12680844962596893, 0.9951980113983154, 0.8884363174438477, 0.8868654370307922, 0.8006849884986877]


In [14]:
test_patchs=[]

## 分类别精度评价

In [22]:
# ev_list=[]
for i in range(0,config.type_num):
    frames=[]
    test_patchs_type=[]
    path_to_write=os.path.join(config.dataset_dir,'test/type'+str(i))
    all_files = os.listdir(path_to_write)
    all_image_files = [fn for fn in all_files if fn.startswith(config.image_fn) and fn.endswith(config.image_type)]#ndwi.png
    for j, fn in enumerate(all_image_files):
        f ,num = readImgs(path_to_write,fn)
        frames.append(f)
    test_patchs_type = DataGenerator(config.input_size, config.output_size, config.patch_size,frames, augmenter = None).all_sequential_patches(config.step_size)
    print('type{} patches number:{}'.format(i,len(test_patchs_type[0])))
    ev2=model.evaluate(test_patchs_type[0],test_patchs_type[1],config.BATCH_SIZE,verbose=1)
    ev_list.append(ev2)

type1 patches number:867
54/55 [============================>.] - ETA: 0s - loss: 0.1297 - dice_loss: 0.1296 - accuracy: 0.9924 - recall: 0.9120 - precision: 0.8539 - IoU: 0.7908

2024-01-08 10:02:27.993365: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-01-08 10:02:28.905234: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-01-08 10:02:29.810982: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-01-08 10:02:30.722829: W tensorflow/stream_executor/gpu/

55/55 [==============================] - 130s 2s/step - loss: 0.1295 - dice_loss: 0.1285 - accuracy: 0.9924 - recall: 0.9133 - precision: 0.8545 - IoU: 0.7923


2024-01-08 10:04:30.603407: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'



type2 patches number:428
26/27 [===========================>..] - ETA: 0s - loss: 0.1248 - dice_loss: 0.1247 - accuracy: 0.9976 - recall: 0.9438 - precision: 0.8347 - IoU: 0.7930

...

2024-01-08 10:08:25.403031: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'



type4 patches number:11
1/1 [==============================] - 0s 125ms/step - loss: 0.0746 - dice_loss: 0.0745 - accuracy: 0.9893 - recall: 0.9698 - precision: 0.8852 - IoU: 0.8615
type5 patches number:151
 9/10 [==========================>...] - ETA: 0s - loss: 0.3501 - dice_loss: 0.3500 - accuracy: 0.9943 - recall: 0.5674 - precision: 0.9438 - IoU: 0.5243

2024-01-08 10:08:35.329871: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-01-08 10:08:35.742266: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-01-08 10:08:36.135985: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'

2024-01-08 10:08:36.528776: W tensorflow/stream_executor/gpu/

10/10 [==============================] - 57s 6s/step - loss: 0.3722 - dice_loss: 0.3976 - accuracy: 0.9945 - recall: 0.5202 - precision: 0.9494 - IoU: 0.4814


2024-01-08 10:09:30.812478: W tensorflow/stream_executor/gpu/asm_compiler.cc:235] Your CUDA software stack is old. We fallback to the NVIDIA driver for some compilation. Update your CUDA version to get the best performance. The ptxas error was: ptxas fatal   : Value 'sm_86' is not defined for option 'gpu-name'



In [27]:
print(ev_list)

[[0.11326627433300018, 0.11252191662788391, 0.992368757724762, 0.9206002950668335, 0.8835827112197876, 0.8875123858451843, 0.8228722214698792], [0.12190033495426178, 0.12098383903503418, 0.9974605441093445, 0.9192798137664795, 0.8614454865455627, 0.8790621757507324, 0.8007123470306396], [0.07600653171539307, 0.07601422816514969, 0.9964860081672668, 0.9427404999732971, 0.9141587615013123, 0.9240355491638184, 0.8629598021507263], [0.08289281278848648, 0.08286094665527344, 0.9885082244873047, 0.9281286597251892, 0.9065663814544678, 0.9172207117080688, 0.8470986485481262], [0.4043097198009491, 0.43480077385902405, 0.9947470426559448, 0.4715380072593689, 0.9832199215888977, 0.5657311081886292, 0.45888274908065796]]


In [ ]:
model.evaluate(test_patchs_type[0],test_patchs_type[1],config.BATCH_SIZE,verbose=1)

In [24]:
print('        loss diclo accur recal preci IoU')
print('total: {:.3f} {:.3f} {:.3f} {:.3f} {:.3f} {:.3f}'.format(ev[0],ev[1],ev[2],ev[3],ev[4],ev[5]))
for i in range(0,config.type_num):
    print('type{}: {:.3f} {:.3f} {:.3f} {:.3f} {:.3f} {:.3f}'.format(i,ev_list[i][0],ev_list[i][1],ev_list[i][2],ev_list[i][3],ev_list[i][4],ev_list[i][5]))

        loss diclo accur recal preci IoU
total: 0.122 0.127 0.995 0.888 0.887 0.801
type1: 0.068 0.067 0.997 0.930 0.944 0.880
type2: 0.129 0.128 0.992 0.913 0.855 0.792
type3: 0.122 0.122 0.998 0.945 0.839 0.798
type4: 0.122 0.121 0.995 0.862 0.923 0.801
type5: 0.075 0.075 0.989 0.970 0.885 0.862
type6: 0.372 0.398 0.995 0.520 0.949 0.481


In [ ]:
del frames,test_patchs,test_patch_type

In [ ]:
%tensorboard --logdir logs